In [ ]:
import urllib.request
import os
import tarfile
print(os.getcwd())

In [ ]:
# 可以手動下載到 現在目錄下的 data/..... 請你手動建立data 目錄
url="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
filepath="data/aclImdb_v1.tar.gz"
if not os.path.isfile(filepath):
    result=urllib.request.urlretrieve(url,filepath)
    print('downloaded:',result)

In [ ]:
# 針對下載的數據進行解壓縮
if not os.path.exists("data/aclImdb"):
    tfile = tarfile.open("data/aclImdb_v1.tar.gz", 'r:gz')
    result=tfile.extractall('data/')

# 1. Import Library

In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


# 資料準備

In [ ]:
#讀取檔案
# <[^>]+> : 找 < 任何文字 不可以是> +多個  >

In [65]:
import re
from nltk.corpus import stopwords
from string import punctuation
stop = stopwords.words('english')

# 拿高分的資料清除位置
def rm_tags(text):
    re_tag = r'<[^>]+>'
    text = text.lower()
    
    text = re.sub(re_tag, '', text)
    text = re.sub(r"didn't","did not", text)
    text = re.sub(r"haven't","have not", text)
    text = re.sub(r"can't","can not", text)
    text = re.sub(r"it's","it is", text)
    text = re.sub(r"won't","will not", text)
    text = re.sub(r"wouldn't","would not",text)
    # 請繼續加入
    
    text = re.sub(r"[&-/()),']", ' ', text) #這特殊符號都去除
    text = re.sub(r"\.", ' ', text) # 點換成一個空白
    text = re.sub(r"\s+", ' ', text) #多空白換成一個空白
    
    text = ' '.join([c for c in text.split() if c not in stop])    
    text = ' '.join([c for c in text.split() if not c.isdigit()])
    text = ' '.join([c for c in text.split() if c not in punctuation])
    return text

In [66]:
text = "FUTZ is the only show preserved from the experimental theatre movement in New York in the 1960s (the origins of Off Off Broadway). Though it's not for everyone, it is a genuinely brilliant, darkly funny, even more often deeply disturbing tale about love, sex, personal liberty, and revenge, a serious morality tale even more relevant now in a time when Congress wants to outlaw gay marriage by trashing our Constitution. The story is not about being gay, though -- it's about love and sex that don't conform to social norms and therefore must be removed through violence and hate. On the surface, it tells the story of a man who falls in love with a pig, but like any great fable, it's not really about animals, it's about something bigger -- stifling conformity in America.<br /><br />The stage version won international acclaim in its original production, it toured the U.S. and Europe, and with others of its kind, influenced almost all theatre that came after it. Luckily, we have preserved here the show pretty much as it was originally conceived, with the original cast and original director, Tom O'Horgan (who also directed HAIR and Jesus Christ Superstar on Broadway).<br /><br />This is not a mainstream, easy-to-take, studio film -- this is an aggressive, unsettling, glorious, deeply emotional, wildly imaginative piece of storytelling that you'll never forget. And it just might change the way you see the world..."
text ="Fair drama/love story movie that focuses on the lives of blue collar people finding new life thru new love.The acting here is good but the film fails in cinematography,screenplay,directing and editing.The story/script is only average at best.This film will be enjoyed by Fonda and De Niro fans and by people who love middle age love stories where in the coartship is on a more wiser and cautious level.   It would also be interesting for people who are interested on the subject matter regarding illiteracy......."
text = "Although I didn't like Stanley & Iris tremendously as a film, I did admire the acting. Jane Fonda and Robert De Niro are great in this movie. I haven't always been a fan of Fonda's work but here she is delicate and strong at the same time. De Niro has the ability to make every role he portrays into acting gold. He gives a great performance in this film and there is a great scene where he has to take his father to a home for elderly people because he can't care for him anymore that will break your heart. I wouldn't really recommend this film as a great cinematic entertainment, but I will say you won't see much bette acting anywhere."
rm_tags(text)

'although like stanley iris tremendously film admire acting jane fonda robert de niro great movie always fan fonda work delicate strong time de niro ability make every role portrays acting gold gives great performance film great scene take father home elderly people care anymore break heart would really recommend film great cinematic entertainment say see much bette acting anywhere'

In [ ]:
import os
def read_files(filetype):
    path = "data/aclImdb/"
    file_list=[]

    positive_path=path + filetype+"/pos/"
    for f in os.listdir(positive_path):
        file_list+=[positive_path+f]
    
    negative_path=path + filetype+"/neg/"
    for f in os.listdir(negative_path):
        file_list+=[negative_path+f]
        
    print('read',filetype, 'files:',len(file_list))
       
    all_labels = ([1] * 12500 + [0] * 12500) 
    
    all_texts  = []
    for fi in file_list:
        with open(fi,encoding='utf8') as file_input:
            all_texts += [rm_tags(" ".join(file_input.readlines()))]
            
    return all_labels,all_texts

In [ ]:
y_train,train_text=read_files("train")

In [ ]:
y_test,test_text=read_files("test")

In [ ]:
#查看正面評價的影評

In [ ]:
train_text[0]

In [ ]:
y_train[0]

In [ ]:
#查看負面評價的影評

In [ ]:
train_text[12499]

In [ ]:
y_train[12499]

# 先讀取所有文章建立字典，限制字典的數量為nb_words=2000

In [ ]:
token = Tokenizer(num_words=2000)
token.fit_on_texts(train_text)

In [ ]:
#Tokenizer屬性

In [ ]:
#fit_on_texts 讀取多少文章

In [ ]:
print(token.document_count)

In [ ]:
print(token.word_index)

# 將每一篇文章的文字轉換一連串的數字
#只有在字典中的文字會轉換為數字

In [ ]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq  = token.texts_to_sequences(test_text)

In [ ]:
print(train_text[0])

In [ ]:
print(x_train_seq[0])

# 讓轉換後的數字長度相同

In [ ]:
#
#文章內的文字，轉換為數字後，每一篇的文章地所產生的數字長度都不同，因為後需要進行類神經網路的訓練，所以每一篇文章所產生的數字長度必須相同
#以下列程式碼為例maxlen=100，所以每一篇文章轉換為數字都必須為100
#bj6eji3t03g/ 2k

In [ ]:
x_train = sequence.pad_sequences(x_train_seq, maxlen=100)
x_test  = sequence.pad_sequences(x_test_seq,  maxlen=100)

In [ ]:
#如果文章轉成數字大於0,pad_sequences處理後，會truncate前面的數字

In [ ]:
print('before pad_sequences length=',len(x_train_seq[0]))
print(x_train_seq[0])

In [ ]:
print('after pad_sequences length=',len(x_train[0]))
print(x_train[0])

In [ ]:
#如果文章轉成數字不足100,pad_sequences處理後，前面會加上0

In [ ]:
print('before pad_sequences length=',len(x_train_seq[1]))
print(x_train_seq[1])

In [ ]:
print('after pad_sequences length=',len(x_train[1]))
print(x_train[1])

# 資料預處理

In [ ]:
token = Tokenizer(num_words=2000)
token.fit_on_texts(train_text)

In [ ]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq  = token.texts_to_sequences(test_text)

In [ ]:
x_train = sequence.pad_sequences(x_train_seq, maxlen=100)
x_test  = sequence.pad_sequences(x_test_seq,  maxlen=100)